In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

Prior stepts to consider
- update Project.xlsx file in "INPUT" folder and run data.ipynb to update publications_list.xlsx

In [2]:
all_projects = pd.read_excel("../../INPUT/Projects.xlsx")

In [3]:
project_numbers_unique = all_projects["Project Number"].unique()
website_list = []
for i in project_numbers_unique:
    website_list.append("https://cordis.europa.eu/project/id/" + str(i))

In [4]:
print(website_list)
print(len(website_list))

['https://cordis.europa.eu/project/id/101136659', 'https://cordis.europa.eu/project/id/101134958', 'https://cordis.europa.eu/project/id/101131588', 'https://cordis.europa.eu/project/id/MR/Y012682/1', 'https://cordis.europa.eu/project/id/2023-00961_VR', 'https://cordis.europa.eu/project/id/345077', 'https://cordis.europa.eu/project/id/360978', 'https://cordis.europa.eu/project/id/101134575', 'https://cordis.europa.eu/project/id/101130249', 'https://cordis.europa.eu/project/id/101145196', 'https://cordis.europa.eu/project/id/101110016', 'https://cordis.europa.eu/project/id/101152592', 'https://cordis.europa.eu/project/id/101135505', 'https://cordis.europa.eu/project/id/101157652', 'https://cordis.europa.eu/project/id/101103474', 'https://cordis.europa.eu/project/id/101162646', 'https://cordis.europa.eu/project/id/101151717', 'https://cordis.europa.eu/project/id/2024-01521', 'https://cordis.europa.eu/project/id/101135051', 'https://cordis.europa.eu/project/id/362378, 362379', 'https://cor

In [5]:
def scrape_peer_reviewed_articles(base_urls):
    # Setup WebDriver
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options)
    all_data = []
    url_count = 0

    try:
        for base_url in base_urls:
            url_count+=1
            print(base_url)
            print(f"url number: {url_count}")
            # Append '/results' to the base URL and navigate to it
            target_url = f"{base_url.rstrip('/')}/results"
            driver.get(target_url)

            wait = WebDriverWait(driver, 1.5)

            # Extract the main project title
            try:
                project_title = wait.until(
                    EC.presence_of_element_located((By.XPATH, "//h1[@class='c-header-project__title']"))
                ).text
            except Exception as e:
                print(f"Could not find project title for {base_url}. Error: {e}")
                project_title = "N/A"

            # Handle Cookie Banner
            try:
                cookie_accept_button = wait.until(
                    EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept')] | //button[contains(text(), 'I agree')]"))
                )
                cookie_accept_button.click()
            except Exception as e:
                print("No cookie banner to handle, proceeding.")

            # Wait for the peer-reviewed section to load
            try:
                peer_review_section = wait.until(
                    EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'c-collapse__header')]//span[contains(text(), 'Peer reviewed articles')]"))
                )

                # Expand the section if it's collapsed
                collapse_link = peer_review_section.find_element(By.XPATH, "./ancestor::div[contains(@class, 'c-collapse')]")
                if "c-collapse__content-visible" not in collapse_link.get_attribute("class"):
                    collapse_button = collapse_link.find_element(By.XPATH, ".//a[@role='button']")
                    driver.execute_script("arguments[0].click();", collapse_button)
                    wait.until(EC.presence_of_element_located((By.XPATH, ".//div[contains(@class, 'c-collapse__content-visible')]")))

                # Find all article entries
                articles = driver.find_elements(By.XPATH, "//div[contains(@class, 'c-collapse__content-visible')]/a")

                for article in articles:
                    # Extract publication details
                    title = article.text
                    link = article.get_attribute("href")
                    details = article.find_element(By.XPATH, "./following-sibling::p")
                    
                    # Extract specific fields
                    author = details.find_element(By.XPATH, ".//span[contains(text(), 'Author(s):')]/following-sibling::br").get_attribute("innerText").split("Author(s):")[-1].strip()
                    published_in = details.find_element(By.XPATH, ".//span[contains(text(), 'Published in:')]/following-sibling::br").get_attribute("innerText").split("Published in:")[-1].strip()
                    doi = details.find_element(By.XPATH, ".//span[contains(text(), 'DOI:')]/following-sibling::br").get_attribute("innerText").split("DOI:")[-1].strip()
                    
                    # Append data to the list
                    all_data.append({
                        "Project Title": project_title,
                        "Base Link": base_url,
                        "Publication Title": title,
                        "Authors": author,
                        "Published In": published_in,
                        "DOI": doi,
                        "Link": link
                    })
                    print(title, link, details, author, published_in, doi)

            except Exception as e:
                print(f"No peer-reviewed articles found for {base_url}. Error: {e}")

    except Exception as e:
        print(f"Error: {e}")
    finally:
        driver.quit()

    # Convert data to DataFrame for better visualization
    df = pd.DataFrame(all_data)
    print(df)

    # Save to a CSV file if needed
    df.to_csv("peer_reviewed_articles_with_titles.csv", index=False)
    print("Scraping complete. Data saved to 'peer_reviewed_articles_with_titles.csv'.")

# Usage
links = [
    "https://cordis.europa.eu/project/id/965196",  # Beispiel-Link 1
]

scrape_peer_reviewed_articles(website_list)

https://cordis.europa.eu/project/id/101136659
url number: 1
No cookie banner to handle, proceeding.
No peer-reviewed articles found for https://cordis.europa.eu/project/id/101136659. Error: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7A54480D5+2992373]
	(No symbol) [0x00007FF7A50DBFD0]
	(No symbol) [0x00007FF7A4F7590A]
	(No symbol) [0x00007FF7A4FC926E]
	(No symbol) [0x00007FF7A4FC955C]
	(No symbol) [0x00007FF7A50127D7]
	(No symbol) [0x00007FF7A4FEF3AF]
	(No symbol) [0x00007FF7A500F584]
	(No symbol) [0x00007FF7A4FEF113]
	(No symbol) [0x00007FF7A4FBA918]
	(No symbol) [0x00007FF7A4FBBA81]
	GetHandleVerifier [0x00007FF7A54A6A2D+3379789]
	GetHandleVerifier [0x00007FF7A54BC32D+3468109]
	GetHandleVerifier [0x00007FF7A54B0043+3418211]
	GetHandleVerifier [0x00007FF7A523C78B+847787]
	(No symbol) [0x00007FF7A50E757F]
	(No symbol) [0x00007FF7A50E2FC4]
	(No symbol) [0x00007FF7A50E315D]
	(No symbol) [0x00007FF7A50D2979]
	BaseThreadInitThunk [0x00007FFF2B2CE8D7+23]
	RtlUserThreadStart [0x00007

In [6]:
df = pd.read_csv("peer_reviewed_articles_with_titles.csv")
df.to_excel("peer_reviewed_articles_with_titles.xlsx")

In [7]:
import pandas as pd
from fuzzywuzzy import fuzz

# Dateien laden
publication_list_path = '../../publications_list.xlsx'
peer_reviewed_path = 'peer_reviewed_articles_with_titles.csv'
output_file_path = 'missing_publications_v3.xlsx'

publications_list = pd.read_excel(publication_list_path)
peer_reviewed = pd.read_csv(peer_reviewed_path)

# Spaltennamen bereinigen und vereinheitlichen
publications_list.columns = publications_list.columns.str.strip().str.lower()
peer_reviewed.columns = peer_reviewed.columns.str.strip().str.lower()

# Spalten-Referenzen
eventual_cordis_col = "eventual cordis link"
base_link_col = "base link"
pub_title_col_list = "publication title"
pub_title_col_csv = "publication title"
project_name_col = "project name"

# Merge: Projekte abgleichen
project_matched = peer_reviewed.merge(
    publications_list[[eventual_cordis_col, project_name_col]].drop_duplicates(),
    how="left",
    left_on=base_link_col,
    right_on=eventual_cordis_col
)

# Aufteilen in bekannte und neue Projekte
new_projects = project_matched[project_matched['project name'].isna()].copy()
known_projects = project_matched[project_matched['project name'].notna()].copy()

# Fehlende Projektnamen in neuen Projekten füllen
new_projects['project name'] = "Unknown Project"

# Fuzzy-Matching-Funktion optimiert
def is_new_publication(title, existing_titles):
    """Prüft, ob ein Titel mindestens 95% Übereinstimmung mit vorhandenen Titeln aufweist."""
    if not existing_titles:
        return True
    return not any(fuzz.ratio(title.lower(), existing_title) >= 95 for existing_title in existing_titles)

# Sammeln fehlender Publikationen
missing_entries = []

# Bekannte Projekte: Titel abgleichen
for project, group in known_projects.groupby('project name'):
    existing_titles = publications_list[
        publications_list['project name'] == project
    ][pub_title_col_list].dropna().str.lower().tolist()

    for _, row in group.iterrows():
        if is_new_publication(row[pub_title_col_csv], existing_titles):
            missing_entries.append(row.to_dict())  # Konvertiere Series zu Dict

# Neue Projekte: Alle Publikationen hinzufügen
missing_entries.extend(new_projects.to_dict('records'))

# Ergebnis DataFrame erstellen
missing_df = pd.DataFrame(missing_entries)

# Auswahl der gewünschten Spalten
output_columns = [
    'project name', base_link_col, pub_title_col_csv,
    "authors", "published in", "link"
]

# Sicherstellen, dass alle Spalten vorhanden sind
for col in output_columns:
    if col not in missing_df.columns:
        missing_df[col] = ""

# Ausgabe vorbereiten
output = missing_df[output_columns].copy()
output['blacklist'] = ""  # Leere Blacklist-Spalte

# Ergebnis speichern
output.to_excel(output_file_path, index=False, sheet_name="Missing Publications")
print(f"Die fehlenden Publikationen wurden in '{output_file_path}' gespeichert.")

C:\Users\Besitzer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Die fehlenden Publikationen wurden in 'missing_publications_v3.xlsx' gespeichert.


In [8]:
print(publications_list.columns)
print(peer_reviewed.columns)


Index(['publication title', 'publication date', 'publication file',
       'project name', 'total project publications', 'project start date',
       'project end date', 'eventual cordis link'],
      dtype='object')
Index(['project title', 'base link', 'publication title', 'authors',
       'published in', 'doi', 'link'],
      dtype='object')
